<a href="https://colab.research.google.com/github/noushsuon/honours-project/blob/edit-carbon-configuration/BERT-Large-Cased.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install comet-ml
!pip install cumulator
!pip install geopy
!pip install GPUtil
!pip install py-cpuinfo
!pip install datasets
!pip install evaluate
!pip install accelerate -U
!pip install codecarbon
!pip install transformers
!pip install tokenizers

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [1]:
import comet_ml
from comet_ml import Experiment
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

import csv
hardwareFile = "/usr/local/lib/python3.10/dist-packages/codecarbon/data/hardware/cpu_power.csv"
cpu_data = ['13th Gen Intel(R) Core(TM) i9-13950HX', 55]
with open(hardwareFile, 'a', newline='') as file:
  writer = csv.writer(file)
  writer.writerow(cpu_data)

comet_ml.init(api_key='uyaIbaxDfy7jK9j7u4KNxeKNg',project_name='comet-bert-experiment')
experiment = comet_ml.Experiment(api_key="uyaIbaxDfy7jK9j7u4KNxeKNg",project_name="comet-bert-experiment",)

COMET INFO: Valid Comet API Key saved in /root/.comet.config (set COMET_CONFIG to change where it is saved).
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
[codecarbon INFO @ 22:02:31] [setup] RAM Tracking...
[codecarbon INFO @ 22:02:31] [setup] GPU Tracking...
[codecarbon INFO @ 22:02:31] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 22:02:31] [setup] CPU Tracking...
[codecarbon WARNING @ 22:02:31] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 22:02:32] CPU Model on constant consumption mode: 13th Gen Intel(R) Core(TM) i9-13950HX
[codecarbon INFO @ 22:02:32] >>> Tracker's metadata:
[codecarbon INFO @ 22:02:32]   Platform system: Linux-5.15.133.1-microsoft-standard-WSL2-x86_64-with-glibc2.35
[codecarbon INFO @ 22:02:32]   Python version: 3.10.12
[codecarbon INFO @ 22:02:32]   CodeCarbon version: 2.3.5
[codecarbon INFO @ 22

In [2]:
hardwareFile = "/usr/local/lib/python3.10/dist-packages/cumulator/hardware_data/cpu.csv"
cpu_data = ['Core i9-13950HX', 'Raptor Lake-HX', '1 / 23','2.2 to 5.5 GHz','BGA 1964 ','10 nm', '36 MB', '55 W', 55]
with open(hardwareFile, 'a', newline='') as file:
  writer = csv.writer(file)
  writer.writerow(cpu_data)

In [3]:
baseFile = "/usr/local/lib/python3.10/dist-packages/cumulator/base.py"
with open(baseFile, 'r') as file:
        content = file.read()
with open(baseFile, 'w') as file:
        file.write(content.replace("src.cumulator", "cumulator"))
from cumulator import base
cumulator = base.Cumulator()
cumulator.on()

CPU recognized: TDP set to 55


In [4]:
from datasets import load_dataset
from transformers import AutoTokenizer

model_checkpoint = "bert-large-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

raw_datasets = load_dataset("squad")
raw_datasets

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [5]:
context = raw_datasets["train"][0]["context"]
question = raw_datasets["train"][0]["question"]

inputs = tokenizer(question, context)
tokenizer.decode(inputs["input_ids"])

'[CLS] To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France? [SEP] Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend " Venite Ad Me Omnes ". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive ( and in a direct line that connects through 3 statues and the Gold Dome ), is a simple, modern stone statue of Mary. [SEP]'

In [6]:
max_length = 384
stride = 128


def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second", #only truncates the context
        stride=stride,
        return_overflowing_tokens=True, #model will store strings ignored when truncating
        return_offsets_mapping=True, #stores the location of tokens in original text
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [7]:
#use previously defined function to preprocess training data
train_dataset = raw_datasets["train"].map(
    preprocess_training_examples,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)
len(raw_datasets["train"]), len(train_dataset) #check the sentences have been tokenised

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

[codecarbon INFO @ 22:02:47] Energy consumed for RAM : 0.000024 kWh. RAM Power : 5.789747714996338 W
[codecarbon INFO @ 22:02:48] Energy consumed for all GPUs : 0.000025 kWh. Total GPU Power : 6.024847096123476 W
[codecarbon INFO @ 22:02:48] Energy consumed for all CPUs : 0.000118 kWh. Total CPU Power : 27.5 W
[codecarbon INFO @ 22:02:48] 0.000167 kWh of electricity used since the beginning.


(87599, 88729)

In [8]:
'''
similar to training data except
  -without start and end positions
  -simplified offset mapping
  -uses example ids
'''
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [9]:
#use function to preprocess validation data
validation_dataset = raw_datasets["validation"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=raw_datasets["validation"].column_names,
)
len(raw_datasets["validation"]), len(validation_dataset)

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

(10570, 10822)

post processing

In [10]:
import evaluate
metric = evaluate.load("squad")

[codecarbon INFO @ 22:03:02] Energy consumed for RAM : 0.000048 kWh. RAM Power : 5.789747714996338 W
[codecarbon INFO @ 22:03:03] Energy consumed for all GPUs : 0.000048 kWh. Total GPU Power : 5.593004541952005 W
[codecarbon INFO @ 22:03:03] Energy consumed for all CPUs : 0.000232 kWh. Total CPU Power : 27.5 W
[codecarbon INFO @ 22:03:03] 0.000327 kWh of electricity used since the beginning.


In [11]:
from tqdm.auto import tqdm
import collections
import numpy as np

n_best = 20
max_answer_length = 30
predicted_answers = []

def get_example(index):
    return tokenized_datasets["test"][index]["text"]

def compute_metrics(start_logits, end_logits, features, examples):
    experiment = comet_ml.get_global_experiment()
    example_to_features = collections.defaultdict(list)

    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []

    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples
    ]
    metric_result = metric.compute(predictions=predicted_answers, references=theoretical_answers)
    if experiment:
        #experiment.set_epoch(epoch)
        experiment.log_metric("f1", metric_result['f1'])
        experiment.log_metric("exact_match", metric_result['exact_match'])
    return metric_result

In [12]:
from transformers import AutoModelForQuestionAnswering
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

[codecarbon INFO @ 22:03:17] Energy consumed for RAM : 0.000071 kWh. RAM Power : 5.789747714996338 W
[codecarbon INFO @ 22:03:18] Energy consumed for all GPUs : 0.000105 kWh. Total GPU Power : 14.162562613431591 W
[codecarbon INFO @ 22:03:18] Energy consumed for all CPUs : 0.000347 kWh. Total CPU Power : 27.5 W
[codecarbon INFO @ 22:03:18] 0.000523 kWh of electricity used since the beginning.
[codecarbon INFO @ 22:03:32] Energy consumed for RAM : 0.000095 kWh. RAM Power : 5.789747714996338 W
[codecarbon INFO @ 22:03:33] Energy consumed for all GPUs : 0.000172 kWh. Total GPU Power : 16.502699951740258 W
[codecarbon INFO @ 22:03:33] Energy consumed for all CPUs : 0.000461 kWh. Total CPU Power : 27.5 W
[codecarbon INFO @ 22:03:33] 0.000728 kWh of electricity used since the beginning.
[codecarbon INFO @ 22:03:47] Energy consumed for RAM : 0.000118 kWh. RAM Power : 5.789747714996338 W
[codecarbon INFO @ 22:03:47] Energy consumed for all GPUs : 0.000232 kWh. Total GPU Power : 14.803054603952

In [13]:
import transformers
from transformers import TrainingArguments
import accelerate

%env COMET_MODE=ONLINE
%env COMET_LOG_ASSETS=TRUE

args = TrainingArguments(
    "bert-large-finetuned-squad-final",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=100,
    fp16=True,
    push_to_hub=True,
    report_to=["comet_ml"],
    logging_strategy="epoch",
    seed=42,
    load_best_model_at_end=False,
    save_total_limit=1,
)

env: COMET_MODE=ONLINE
env: COMET_LOG_ASSETS=TRUE


In [14]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(
[codecarbon INFO @ 22:04:22] Energy consumed for RAM : 0.000174 kWh. RAM Power : 5.789747714996338 W
[codecarbon INFO @ 22:04:22] Energy consumed for all GPUs : 0.000384 kWh. Total GPU Power : 35.14476455674528 W
[codecarbon INFO @ 22:04:22] Energy consumed for all CPUs : 0.000841 kWh. Total CPU Power : 27.5 W
[codecarbon INFO @ 22:04:22] 0.001399 kWh of electricity used since the beginning.
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: -------------------------------------

Epoch,Training Loss,Validation Loss
1,1.068500,No log
2,0.596900,No log
3,0.342000,No log


[codecarbon INFO @ 22:04:42] Energy consumed for RAM : 0.000024 kWh. RAM Power : 5.789747714996338 W
[codecarbon INFO @ 22:04:42] Energy consumed for all GPUs : 0.000439 kWh. Total GPU Power : 105.37975844797595 W
[codecarbon INFO @ 22:04:42] Energy consumed for all CPUs : 0.000115 kWh. Total CPU Power : 27.5 W
[codecarbon INFO @ 22:04:42] 0.000578 kWh of electricity used since the beginning.
[codecarbon INFO @ 22:04:57] Energy consumed for RAM : 0.000048 kWh. RAM Power : 5.789747714996338 W
[codecarbon INFO @ 22:04:57] Energy consumed for all GPUs : 0.000922 kWh. Total GPU Power : 115.87215021983134 W
[codecarbon INFO @ 22:04:57] Energy consumed for all CPUs : 0.000229 kWh. Total CPU Power : 27.5 W
[codecarbon INFO @ 22:04:57] 0.001199 kWh of electricity used since the beginning.
[codecarbon INFO @ 22:05:12] Energy consumed for RAM : 0.000072 kWh. RAM Power : 5.789747714996338 W
[codecarbon INFO @ 22:05:12] Energy consumed for all GPUs : 0.001403 kWh. Total GPU Power : 115.53002239881

TrainOutput(global_step=33276, training_loss=0.6691069357067594, metrics={'train_runtime': 10081.3766, 'train_samples_per_second': 26.404, 'train_steps_per_second': 3.301, 'total_flos': 1.8540729608518195e+17, 'train_loss': 0.6691069357067594, 'epoch': 3.0})

In [15]:
predictions, _, _ = trainer.predict(validation_dataset)
start_logits, end_logits = predictions
compute_metrics(start_logits, end_logits, validation_dataset, raw_datasets["validation"])

  0%|          | 0/10570 [00:00<?, ?it/s]

{'exact_match': 84.10596026490066, 'f1': 91.09175190560305}

In [16]:
cumulator.off()
cumulator.on()

In [17]:
cumulator.display_carbon_footprint()

########
Overall carbon footprint: 2.76e+01 gCO2eq
########
Carbon footprint due to computations: 2.76e+01 gCO2eq
Carbon footprint due to communications: 0.00e+00 gCO2eq
This carbon footprint is equivalent to 7.82e-04 square meters of u.s. forests preserved from conversion to cropland in one year.


In [18]:
trainer.push_to_hub(commit_message="Training complete")

CommitInfo(commit_url='https://huggingface.co/noushsuon/bert-large-finetuned-squad-final/commit/5b41e23cffa51ba97912c400d545a8a3e63f66b3', commit_message='Training complete', commit_description='', oid='5b41e23cffa51ba97912c400d545a8a3e63f66b3', pr_url=None, pr_revision=None, pr_num=None)

In [19]:
from torch.utils.data import DataLoader
from transformers import default_data_collator

train_dataset.set_format("torch")
validation_set = validation_dataset.remove_columns(["example_id", "offset_mapping"])
validation_set.set_format("torch")

train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    collate_fn=default_data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    validation_set, collate_fn=default_data_collator, batch_size=8
)

In [20]:
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-large-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

In [22]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [23]:
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [24]:
from huggingface_hub import Repository, get_full_repo_name
model_name = "bert-large-finetuned-squad-final"
repo_name = get_full_repo_name(model_name)
repo_name

'noushsuon/bert-large-finetuned-squad-final'

In [25]:
from huggingface_hub import Repository
output_dir = "bert-large-finetuned-squad-final-accelerate" + model_checkpoint

repo = Repository(output_dir, clone_from=repo_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/noushsuon/bert-large-finetuned-squad-final into local empty directory.


Download file model.safetensors:   0%|          | 16.0k/1.24G [00:00<?, ?B/s]

Download file training_args.bin: 100%|##########| 4.49k/4.49k [00:00<?, ?B/s]

Clean file training_args.bin:  22%|##2       | 1.00k/4.49k [00:00<?, ?B/s]

Clean file model.safetensors:   0%|          | 1.00k/1.24G [00:00<?, ?B/s]

In [26]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for step, batch in enumerate(train_dataloader):
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    start_logits = []
    end_logits = []
    accelerator.print("Evaluation!")
    for batch in tqdm(eval_dataloader):
        with torch.no_grad():
            outputs = model(**batch)

        start_logits.append(accelerator.gather(outputs.start_logits).cpu().numpy())
        end_logits.append(accelerator.gather(outputs.end_logits).cpu().numpy())

    start_logits = np.concatenate(start_logits)
    end_logits = np.concatenate(end_logits)
    start_logits = start_logits[: len(validation_dataset)]
    end_logits = end_logits[: len(validation_dataset)]

    metrics = compute_metrics(
        start_logits, end_logits, validation_dataset, raw_datasets["validation"]
    )

    for key, value in metrics.items():
        experiment.log_metric(key, value)

    print(f"epoch {epoch}:", metrics)

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        repo.push_to_hub(
            commit_message=f"Training in progress epoch {epoch}", blocking=False
        )

  0%|          | 0/33276 [00:00<?, ?it/s]

Evaluation!


  0%|          | 0/1353 [00:00<?, ?it/s]

  0%|          | 0/10570 [00:00<?, ?it/s]

epoch 0: {'exact_match': 83.72753074739829, 'f1': 90.79481511533518}
Evaluation!


...

  0%|          | 0/10570 [00:00<?, ?it/s]

epoch 1: {'exact_match': 84.21948912015137, 'f1': 91.1399484859757}
Evaluation!


  0%|          | 0/1353 [00:00<?, ?it/s]

  0%|          | 0/10570 [00:00<?, ?it/s]

epoch 2: {'exact_match': 83.99243140964995, 'f1': 91.12161896516572}


In [27]:
accelerator.wait_for_everyone()
unwrapped_model = accelerator.unwrap_model(model)
unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)

In [28]:
experiment.end()

In [29]:
cumulator.off()
cumulator.display_carbon_footprint()

########
Overall carbon footprint: 5.26e+01 gCO2eq
########
Carbon footprint due to computations: 5.26e+01 gCO2eq
Carbon footprint due to communications: 0.00e+00 gCO2eq
This carbon footprint is equivalent to 1.95e-02 liters of diesel consumed.


In [30]:
cumulator.time_list

[10688.248741149902, 9673.390804052353]